# Invoke Model Endpoint:
## 2. Client to Endpoint using Boto3 SDK
Previous activity: completing the model training and model endpoint deployment.

In [1]:
import boto3
import pandas as pd

In [2]:
# Start a boto3 session with "user"
boto3_session = boto3.Session(
    profile_name="user", region_name="ap-southeast-1")

In [3]:
# Display the available services from boto3 and find if sagemaker-runtime is included
print(boto3_session.get_available_services())

['accessanalyzer', 'account', 'acm', 'acm-pca', 'alexaforbusiness', 'amp', 'amplify', 'amplifybackend', 'amplifyuibuilder', 'apigateway', 'apigatewaymanagementapi', 'apigatewayv2', 'appconfig', 'appconfigdata', 'appflow', 'appintegrations', 'application-autoscaling', 'application-insights', 'applicationcostprofiler', 'appmesh', 'apprunner', 'appstream', 'appsync', 'athena', 'auditmanager', 'autoscaling', 'autoscaling-plans', 'backup', 'backup-gateway', 'batch', 'braket', 'budgets', 'ce', 'chime', 'chime-sdk-identity', 'chime-sdk-meetings', 'chime-sdk-messaging', 'cloud9', 'cloudcontrol', 'clouddirectory', 'cloudformation', 'cloudfront', 'cloudhsm', 'cloudhsmv2', 'cloudsearch', 'cloudsearchdomain', 'cloudtrail', 'cloudwatch', 'codeartifact', 'codebuild', 'codecommit', 'codedeploy', 'codeguru-reviewer', 'codeguruprofiler', 'codepipeline', 'codestar', 'codestar-connections', 'codestar-notifications', 'cognito-identity', 'cognito-idp', 'cognito-sync', 'comprehend', 'comprehendmedical', 'co

In [4]:
# Sagemaker-runtime client
client = boto3_session.client(
    service_name='sagemaker-runtime', region_name='ap-southeast-1')

In [5]:
# Prepare the input data sample
input_sample_1 = "1.0, 1.0, 0.0, 1.0, 0.0, 2083.0, 3150.0, 128.0, 360.0, 1.0, 0.0, 1.0, 0.0"  # true value: 1
input_sample_2 = "1.0, 0.0, 0.0, 0.0, 0.0, 2333.0, 1451.0, 102.0, 480.0, 0.0, 0.0, 0.0, 1.0"  # true value: 0
input_sample_3 = "1.0, 1.0, 1.0, 1.0, 0.0, 6325.0, 0.0, 175.0, 360.0, 1.0, 0.0, 1.0, 0.0"  # true value: 1

In [6]:
# Invoke the prediction (1 input)
pred = client.invoke_endpoint(EndpointName="xgboost-loan",
                              Body=input_sample_1.encode('utf-8'),
                              ContentType='text/csv')
pred

{'ResponseMetadata': {'RequestId': '2d7df732-85af-405b-9335-de84f80f2c2b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2d7df732-85af-405b-9335-de84f80f2c2b',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Sat, 22 Jan 2022 11:16:13 GMT',
   'content-type': 'text/csv; charset=utf-8',
   'content-length': '3'},
  'RetryAttempts': 0},
 'ContentType': 'text/csv; charset=utf-8',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x2415fa7c5b0>}

In [7]:
pred = pred['Body'].read().decode('utf-8')
print(pred)

1.0


In [8]:
# Invoke the prediction (multiple inputs)
pred = client.invoke_endpoint(EndpointName="xgboost-loan",
                              Body=('\n'.join([input_sample_1,
                                               input_sample_2,
                                               input_sample_3]).encode('utf-8')),
                              ContentType='text/csv')

pred = pred['Body'].read().decode('utf-8')
print(pred)

1.0,0.0,1.0


In [9]:
# Load test data
test_data = pd.read_csv("D:/3. Projects/AWS Sagemaker/others/loan_test_data.csv")
print(f"The length of the test data is {len(test_data)}")

test_input = []
with open("D:/3. Projects/AWS Sagemaker/others/loan_test_data.csv", 'r') as f:
    f.readline()
    test_input = f.readlines()

The length of the test data is 96


In [10]:
# Batch Prediction
def transform(data):
    feature = data.split(',')

    if feature[1] == "Male":
        feature[1] = "1"
    else:
        feature[1] = "0"

    if feature[2] == "Yes":
        feature[2] = "1"
    else:
        feature[2] = "0"

    if feature[3] == "3+":
        feature[3] = "3"
    else:
        feature[3] = feature[3]

    if feature[4] == "Graduate":
        feature[4] = "1"
    else:
        feature[4] = "0"

    if feature[5] == "Yes":
        feature[5] = "1"
    else:
        feature[5] = "0"

    feature.extend(["0", "0", "0"])

    if feature[11] == "Rural":
        feature[12] = "1"
    elif feature[11] == "Semiurban":
        feature[13] = "1"
    elif feature[11] == "Urban":
        feature[14] = "1"
    feature.pop(11)

    feature = ','.join(feature[1:])

    return feature


def batch_prediction(test_input, batch_size):
    predictions = []

    # Transform input data
    test_input = [transform(row.strip()) for row in test_input]

    for i in range(0, len(test_input), batch_size):
        print(f"predicting row {i}")

        # Predict the test data
        pred = client.invoke_endpoint(
            EndpointName="xgboost-loan",
            Body=("\n".join(test_input[i: i + batch_size]).encode('utf-8')),
            ContentType='text/csv'
        )
        pred = pred['Body'].read().decode('utf-8')
        pred = pred.split(',')
        predictions += pred

    return predictions

In [11]:
# Batch prediction on the test data
pred = batch_prediction(test_input, 20)
pred

predicting row 0
predicting row 20
predicting row 40
predicting row 60
predicting row 80


['1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '0.0',
 '1.0']

In [12]:
# Number of predictions
print(f"Number of predictions: {len(pred)}")

Number of predictions: 96


Finally, delete the endpoint (if not used anymore)